In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

In [66]:
fs_df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_51275_2024-05-14~2025-05-14.xlsx')
na_df = pd.read_csv('/Users/seanyu_mac/Downloads/네이버스마트스토어 공구 - 시트1 (6).csv')

In [78]:
fs_df.loc[~fs_df['주문 상태'].isin(['취소완료']), '정산가'].sum()

6105000

In [32]:
fs_df = fs_df[fs_df['상품 번호'] == 51275]

In [80]:
na_df['박스수량'] = na_df['옵션정보'].apply(lambda x: x.split('박스')[0].split('X')[1].strip())
na_df['박스수량'] = na_df['박스수량'].astype(int)

In [82]:
con2 = na_df['박스수량'] == 2
con4 = na_df['박스수량'] == 4
con8 = na_df['박스수량'] == 8
con12 = na_df['박스수량'] == 12

na_df['price'] = 0
na_df.loc[con2, '판매가'] = 57000
na_df.loc[con4, '판매가'] = 96000
na_df.loc[con8, '판매가'] = 165000
na_df.loc[con12, '판매가'] = 239000


In [84]:
na_df['매출'] = na_df['판매가'] * na_df['수량']

In [48]:
na_df['주문일시'] = pd.to_datetime(na_df['주문일시'], format='mixed')
na_df['주문일시'] = na_df['주문일시'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [52]:
na_df.columns

Index(['상품주문번호', '주문번호', '주문일시', '주문상태', '배송속성', '풀필먼트사(주문 기준)', '클레임상태',
       '수량클레임 여부', '상품번호', '상품명', '옵션정보', '수량', '구매자명', '구매자ID', '수취인명',
       '구독신청회차', '구독진행회차', '박스수량', 'price', '판매가', '매출'],
      dtype='object')

In [54]:
fs_df['결제 완료일'] = pd.to_datetime(fs_df['결제 완료일'])
fs_df['결제 완료일'] = fs_df['결제 완료일'].apply(lambda x: x.strftime('%Y-%m-%d'))
fs_df['박스수량'] = fs_df['옵션명'].apply(lambda x: x.split('박스')[0].split('X')[1].strip())
fs_df['박스수량'] = fs_df['박스수량'].astype(int)

In [56]:
fs_df = fs_df[['결제 번호', '상품 번호', '주문 번호', '결제 완료일', '주문 상태', '상품명', '옵션명', '수량', '박스수량', '정산가']]
fs_df['구분'] = '친한스토어'

In [58]:
na_df = na_df[['주문번호', '상품주문번호', '상품번호', '주문일시', '주문상태', '상품명', '옵션정보', '수량', '박스수량', '매출']]
na_df['구분'] = '네이버스토어'
na_df.rename(columns={'수량' : '주문수량'}, inplace=True)

In [60]:
fs_df.columns = na_df.columns
df = pd.concat([fs_df, na_df])
df['매출'] = df['매출'].astype(int)

In [62]:
pivot_df = pd.DataFrame(pd.pivot_table(df, index=['주문일시', '옵션정보'], values=['매출', '주문수량', '박스수량'], aggfunc='sum', margins=True, margins_name='합계'))
pivot_df.applymap(lambda x: f'{x:,}')

매출 박스수량 주문수량
주문일시       옵션정보                              
2025-05-08 30포X12박스       1,195,000   60    5
           30포X2박스          513,000   18    9
           30포X4박스        1,056,000   44   11
           30포X8박스          990,000   48    6
2025-05-09 30포X12박스         956,000   48    4
           30포X2박스          171,000    6    3
           30포X4박스          672,000   28    7
           30포X8박스          165,000    8    1
2025-05-10 30포X2박스           57,000    2    1
           30포X4박스           96,000    4    1
           30포X8박스          330,000   16    2
           선택: 30포X12박스     478,000   24    2
           선택: 30포X2박스      114,000    4    2
           선택: 30포X4박스      480,000   20    5
           선택: 30포X8박스      495,000   24    3
2025-05-11 선택: 30포X12박스   1,434,000   72    6
           선택: 30포X2박스      399,000   14    7
           선택: 30포X4박스      864,000   36    9
           선택: 30포X8박스      660,000   32    4
2025-05-12 30포X2박스           57,000    2    1
           선택: 30포X12박스     478,000   24    2
           선택: 30포X2박스      570,000   20   10
           선택: 30포X4박스      672,000   28    7
2025-05-13 선택: 30포X12박스   1,434,000   72    6
           선택: 30포X2박스      399,000   14    7
           선택: 30포X4박스      288,000   12    3
           선택: 30포X8박스      825,000   40    5
2025-05-14 선택: 30포X2박스       57,000    2    1
           선택: 30포X4박스      192,000    8    2
           선택: 30포X8박스      165,000    8    1
합계                       16,262,000  738  133

In [64]:
pivot_df = pd.DataFrame(pd.pivot_table(df, index=['주문일시', '구분'], values=['매출', '주문수량', '박스수량'], aggfunc='sum', margins=True, margins_name='합계'))
pivot_df.applymap(lambda x: f'{x:,}')

매출 박스수량 주문수량
주문일시       구분                          
2025-05-08 친한스토어    3,754,000  170   31
2025-05-09 친한스토어    1,964,000   90   15
2025-05-10 네이버스토어   1,567,000   72   12
           친한스토어      483,000   22    4
2025-05-11 네이버스토어   3,357,000  154   26
2025-05-12 네이버스토어   1,720,000   72   19
           친한스토어       57,000    2    1
2025-05-13 네이버스토어   2,946,000  138   21
2025-05-14 네이버스토어     414,000   18    4
합계                 16,262,000  738  133